In [1]:
## Load context
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import re
import os
import pandas as pd
import numpy as np
hostname = os.uname()[1]

from pyspark.sql import SparkSession

# BEN'S MASTER: 192.168.2.87
# OUR MASTER:   192.168.2.203

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName(f"load_local_comments; hostname: {hostname}")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.uiWebUrl

'http://host-192-168-2-247-ldsa:4040'

In [13]:
import pyspark.sql.functions as f

df_pd = pd.read_json("sample_reddit_comments.json", lines=True)
df = spark_session.createDataFrame(df_pd).cache()

In [28]:
from pyspark.sql import functions as F

udf_strip_lower = F.udf(lambda comment: (re.sub(r'\W+', ' ', comment).lower().strip()), 'string')
df_1 = df.select('body').withColumn('body', udf_strip_lower('body'))

#df.select('body').withColumn('body', f.lower(f.col('body'))).take(5)

In [38]:
df_2 = df_1.withColumn('word', f.explode(f.split(f.col('body'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

df_2.show()

+----+-----+
|word|count|
+----+-----+
| the|10290|
|   i| 7339|
|  to| 7062|
|   a| 6537|
| and| 5651|
|  it| 4817|
| you| 4567|
|  of| 4377|
|that| 3924|
|  is| 3633|
|  in| 3215|
|   s| 2826|
| for| 2696|
|   t| 2606|
|this| 2385|
| but| 1917|
|  on| 1878|
|have| 1844|
|  be| 1813|
|with| 1793|
+----+-----+
only showing top 20 rows

